### Set Seed Cố Định

import os
import torch
import random
import numpy as np
import time

def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Đặt seed cố định của đội bạn
seed_everything(42) # [cite: 491]
print("Seed set to 42 for reproducibility.")

## Nạp Mô Hình và Tài Nguyên

In [ ]:
from ultralytics import YOLOWorld

start_load = time.time()

model_path = "/code/saved_models/best.pt"
model = YOLOWorld(model_path)
model.set_classes(["target"])
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
model.eval()

end_load = time.time()
print(f"Model Load Time (ms): {int((end_load - start_load) * 1000)}")

## Đọc Nội Dung Các Test Case

In [ ]:
import glob

DATA_ROOT = "/data/samples"
test_cases = glob.glob(os.path.join(DATA_ROOT, "*", "drone_video.mp4"))
print(f"Found {len(test_cases)} test videos.")

## Thực Hiện Dự Đoán và In Ra Kết Quả/Thời Gian

In [ ]:
import pandas as pd
import cv2

all_predicted_time = [] 
all_result = []         

OUTPUT_TIME_FILE = "/result/jupyter_time_submission.csv"
OUTPUT_JSON_FILE = "/result/jupyter_submission.json" 

def write_predict_file(results, path):
    submission = []
    for vid, bboxes in results:
        submission.append({
            "video_id": vid,
            "detections": [{"bboxes": bboxes}] if bboxes else []
        })
    with open(path, "w") as f:
        json.dump(submission, f, indent=2, ensure_ascii=False)
        
def write_time_file(times, path):
    df = pd.DataFrame(times, columns=["id", "answer", "time"])
    df.to_csv(path, index=False)


for file_path in test_cases:
    video_id = os.path.basename(os.path.dirname(file_path)) # Trích xuất ID
    
    cap = cv2.VideoCapture(file_path)
    bboxes_per_frame = []
    frame_idx = 0
    
    t1 = time.time()
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
            
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        results = model.predict(
            source=frame_rgb,
            conf=0.35, iou=0.55, imgsz=896, verbose=False, device=device
        )
        
        result_box = None
        for r in results:
            if r.boxes is not None and len(r.boxes) > 0:
                box_xyxy = r.boxes.xyxy.cpu().numpy()[0]
                x1, y1, x2, y2 = map(int, box_xyxy)
                result_box = [x1, y1, x2, y2]
                break
        
        if result_box is not None:
            x1, y1, x2, y2 = result_box
            bboxes_per_frame.append({
                "frame": frame_idx,
                "x1": x1, "y1": y1, "x2": x2, "y2": y2
            })
            
        frame_idx += 1
        
    cap.release()
    
    t2 = time.time()
    predicted_time = int((t2 - t1) * 1000)
    all_predicted_time.append((video_id, len(bboxes_per_frame), predicted_time))
    all_result.append((video_id, bboxes_per_frame))

write_predict_file(all_result, OUTPUT_JSON_FILE) 
write_time_file(all_predicted_time, OUTPUT_TIME_FILE)

print(f"Jupyter results saved to {OUTPUT_JSON_FILE} and {OUTPUT_TIME_FILE}")